In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "random_walk"
    cd(joinpath(@__DIR__, "../../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMol.Algorithms: RandomWalkMetropolis, simulate!
using MorphoMolNotebooks: calculate_initial_temperature, add_to_simulation_minima!, has_hard_sphere_overlap, get_random_configuration_without_overlap_in_bounds, is_configuration_equivalent_to_mcc, is_equivalent_configuration, get_mccs, minimal_mcc, get_energies_of_mccs
using Random
using Distances
using JLD2
using GeometryBasics
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ MorphoMol
  1 dependency successfully precompiled in 3 seconds. 258 already precompiled.
Precompiling MorphoMolNotebooks
  ✓ MorphoMolNotebooks
  1 dependency successfully precompiled in 5 seconds. 283 already precompiled.


In [3]:
function solvation_free_energy_and_measures_in_bounds(x::Vector{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, bounds::Float64, delaunay_eps::Float64)
    if any(0.0 >= e || e >= bounds for e in x[1:3:end]) || any(0.0 >= e || e >= bounds for e in x[2:3:end]) || any(0.0 >= e || e >= bounds for e in x[3:3:end])
        return Inf, [Inf, Inf, Inf, Inf, Inf]
    end
    measures = MorphoMol.Energies.get_geometric_measures_and_overlap_value(x, 1, radii, rs, overlap_jump, overlap_slope, delaunay_eps)
    sum(measures .* [prefactors; 1.0]), Dict{String,Any}("Vs" => measures[1], "As" => measures[2], "Cs" => measures[3], "Xs" => measures[4], "OLs" => measures[5])
end

function solvation_free_energy_in_bounds(x::Vector{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, bounds::Float64, delaunay_eps::Float64)
    if any(0.0 >= e || e >= bounds for e in x[1:3:end]) || any(0.0 >= e || e >= bounds for e in x[2:3:end]) || any(0.0 >= e || e >= bounds for e in x[3:3:end])
        return Inf
    end
    measures = MorphoMol.Energies.get_geometric_measures_and_overlap_value(x, 1, radii, rs, overlap_jump, overlap_slope, delaunay_eps)
    sum(measures .* [prefactors; 1.0])
end

function solvation_free_energy(x::Vector{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, delaunay_eps::Float64)
    measures = MorphoMol.Energies.get_geometric_measures_and_overlap_value(x, 1, radii, rs, overlap_jump, overlap_slope, delaunay_eps)
    sum(measures .* [prefactors; 1.0])
end

perturb_all(x, Σ) = x .+ (randn(length(x)) .* Σ)

function perturb_single_randomly_chosen(x, σ_t)
    x_cand = deepcopy(x)
    i  = rand(0:(length(x)÷3)-1)
    x_cand[(i*3)+1:(i*3)+3] = x_cand[(i*3)+1:(i*3)+3] .+ (randn(3) .* σ_t)
    x_cand
end

perturb_single_randomly_chosen (generic function with 1 method)

In [71]:
rs = 0.2
eta = 0.475

template_centers = [0.0, 0.0, 0.0]
template_radii = [1.0]
n = 4
bounds = 3.5
x_init = get_random_configuration_without_overlap_in_bounds(n, bounds)
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n]...)

pf = MorphoMol.Energies.get_prefactors(rs, eta)

T_init = calculate_initial_temperature(n, rs, pf)
T = T_init * 4.0
println("T = $T")

iterations = 100000

overlap_slope = 500.0
overlap_jump = 0.0

σ_t = rs
Σ = vcat([[σ_t, σ_t, σ_t] for _ in 1:n]...)

energy(x) = solvation_free_energy_in_bounds(x, radii, rs, pf, overlap_jump, overlap_slope, bounds, 100.0)
perturbation(x) = perturb_single_randomly_chosen(x, σ_t)

rwm = RandomWalkMetropolis(
    energy,
    perturbation,
    1/T,
)
x, E, alpha = MorphoMol.Algorithms.simulate!(rwm, x_init, iterations);
alpha

T = 0.4408062904403778


0.38349

In [72]:
m_mcc = minimal_mcc(4, rs, pf)
energy(x), solvation_free_energy(m_mcc.centers .* 1.0015, radii, rs, pf, 0.0, 0.0, 1000.0)

(3250.9237642849266, 3247.149971050299)

In [73]:
fig = Figure()
sc1 = LScene(fig[1,1])
for c in [e for e in eachcol(reshape(x_init, (3, length(x_init)÷3)))]
    mesh!(sc1, HyperSphere(Point3f(c), Float32(1.0)), color = :blue)
    mesh!(sc1, HyperSphere(Point3f(c), Float32(1.0 + rs)), color = RGBAf(0.0, 0.5, 0.3, 0.25))
end

sc2 = LScene(fig[1,2])
for c in [e for e in eachcol(reshape(x, (3, length(x)÷3)))]
    mesh!(sc2, HyperSphere(Point3f(c), Float32(1.0)), color = :green)
    mesh!(sc2, HyperSphere(Point3f(c), Float32(1.0 + rs)), color = RGBAf(0.0, 0.5, 0.3, 0.25))
end

sc3 = LScene(fig[1,3])
for c in [e for e in eachcol(reshape(m_mcc.centers, (3, length(m_mcc.centers)÷3)))]
    mesh!(sc3, HyperSphere(Point3f(c), Float32(1.0)), color = :red)
    mesh!(sc3, HyperSphere(Point3f(c), Float32(1.0 + rs)), color = RGBAf(0.0, 0.5, 0.3, 0.25))
end

display(fig)

GLMakie.Screen(...)